In [12]:
from dcmrtstruct2nii import dcmrtstruct2nii, list_rt_structs



In [9]:
import os
import numpy as np
import SimpleITK as sitk
import pydicom
import nibabel as nib

def dicom_to_nifti(dicom_dir, output_dir):
    """ Converte una cartella di file DICOM in NIfTI usando SimpleITK """
    os.makedirs(output_dir, exist_ok=True)
    
    dicom_images = []
    
    # Carica i file DICOM in una lista
    for dicom_file in os.listdir(dicom_dir):
        if dicom_file.endswith('.dcm'):
            dicom_path = os.path.join(dicom_dir, dicom_file)
            dicom_data = pydicom.dcmread(dicom_path)
            
            # Verifica se il dataset contiene i dati di pixel
            if 'PixelData' not in dicom_data:
                print(f"Il file {dicom_file} non contiene dati di pixel. Ignorato.")
                continue  # Salta i file che non contengono dati di pixel
                
            dicom_images.append(dicom_data)
    
    # Verifica se abbiamo trovato immagini DICOM valide
    if len(dicom_images) == 0:
        raise ValueError("Nessun file DICOM valido trovato!")
    
    # Ordina le immagini DICOM in base a 'ImagePositionPatient' (per immagini 3D) o 'InstanceNumber' (per immagini 2D)
    try:
        dicom_images.sort(key=lambda x: float(x.ImagePositionPatient[2]))
    except AttributeError:
        dicom_images.sort(key=lambda x: int(x.InstanceNumber))
        print("Immagini 2D rilevate, ordinamento basato su InstanceNumber")
    
    # Ottieni i dati di pixel
    pixel_arrays = [dicom.pixel_array for dicom in dicom_images]
    
    # Crea un array NumPy 3D
    image_array = np.stack(pixel_arrays, axis=-1)
    
    # Crea un'immagine SimpleITK a partire dal NumPy array
    sitk_image = sitk.GetImageFromArray(image_array)
    
    # Imposta l'affine (trasformazione spaziale) basata sui metadati DICOM
    if 'PixelSpacing' in dicom_images[0]:
        spacing = tuple([float(dicom_images[0].PixelSpacing[0]), 
                         float(dicom_images[0].PixelSpacing[1]), 
                         float(dicom_images[0].SliceThickness)])
        sitk_image.SetSpacing(spacing)
    else:
        # Usa una spacing di default se non disponibile
        sitk_image.SetSpacing((1.0, 1.0, 1.0))
    
    # Salva l'immagine NIfTI
    nifti_filename = os.path.join(output_dir, 'output.nii.gz')
    sitk.WriteImage(sitk_image, nifti_filename)
    
    print(f"File NIfTI salvato in: {nifti_filename}")
    
    return sitk_image

def contourSequence2Image(cs, reference_image, spacing=(1, 1, 1)):
    """ Converti sequenza di contorni in un'immagine usando SimpleITK """
    mask = np.zeros(reference_image.GetSize(), dtype=np.uint8)
    
    # Itera sui contorni e crea una maschera
    for contour in cs.ContourSequence:
        for i in range(0, len(contour.ContourData), 3):
            # Estrai coordinate del contorno
            x, y, z = contour.ContourData[i:i+3]
            
            # Proietta il contorno nello spazio 3D (supponiamo che il contorno sia sul piano Z)
            ix, iy, iz = int(x), int(y), int(z)
            
            # Aggiungi il contorno alla maschera
            if 0 <= ix < mask.shape[0] and 0 <= iy < mask.shape[1] and 0 <= iz < mask.shape[2]:
                mask[ix, iy, iz] = 1
    
    # Crea un'immagine SimpleITK a partire dalla maschera
    mask_image = sitk.GetImageFromArray(mask)
    mask_image.SetSpacing(spacing)
    
    return mask_image

def dicom_rt_to_nifti(dicom_dir, rtstruct_file, output_dir):
    """ Pipeline completa per convertire DICOM e RTSTRUCT in NIfTI """
    os.makedirs(output_dir, exist_ok=True)
    
    # Carica serie DICOM e ottieni immagine di riferimento NIfTI
    reference_image = dicom_to_nifti(dicom_dir, output_dir)  # Usa la funzione dicom_to_nifti per ottenere l'immagine
    
    # Carica il file RTSTRUCT
    rtstruct = pydicom.dcmread(rtstruct_file)
    
    # Elabora ogni ROI
    for contour_sequence in rtstruct.ROIContourSequence:
        roi_number = contour_sequence.ReferencedROINumber
        
        # Trova il nome della ROI
        roi_name = next(
            (roi.ROIName for roi in rtstruct.StructureSetROISequence 
             if roi.ROINumber == roi_number), 
            f"ROI_{roi_number}"
        )
        
        print(f"Elaborazione ROI: {roi_name}")
        
        try:
            # Converti la sequenza dei contorni in un'immagine
            mask_image = contourSequence2Image(contour_sequence, reference_image)
            
            # Salva come NIfTI
            output_path = os.path.join(output_dir, f"{roi_name}_mask.nii.gz")
            sitk.WriteImage(mask_image, output_path)
            
            print(f"Maschera salvata per {roi_name}: {output_path}")
        
        except Exception as e:
            print(f"Errore durante l'elaborazione di {roi_name}: {e}")
    
    print("Conversione completata!")


In [10]:
# Utilizzo dello script
dicom_dir = "/home/mario/NasPersonale/Progetti2/EUCAIM/DATA/RT-STRUCT/DCM/ID_1"  # Path alla directory della serie DICOM
rtstruct_file = "/home/mario/NasPersonale/Progetti2/EUCAIM/DATA/RT-STRUCT/1-1.dcm"  # Path al file RTSTRUCT
output_dir = "./prova"  # Directory di output

# dcmrtstruct2nii(rtstruct_file,dicom_dir , output_dir)


In [11]:
dicom_rt_to_nifti(dicom_dir, rtstruct_file, output_dir)


Il file 1.2.276.0.7230010.3.1.4.1381411976.307012.1726490792.903291.dcm non contiene dati di pixel. Ignorato.
File NIfTI salvato in: ./prova/output.nii.gz
Elaborazione ROI: BODY
Maschera salvata per BODY: ./prova/BODY_mask.nii.gz
Elaborazione ROI: Brain
Maschera salvata per Brain: ./prova/Brain_mask.nii.gz
Elaborazione ROI: Brain Stem
Maschera salvata per Brain Stem: ./prova/Brain Stem_mask.nii.gz
Elaborazione ROI: Brain Stem PV
Maschera salvata per Brain Stem PV: ./prova/Brain Stem PV_mask.nii.gz
Elaborazione ROI: COUCH
Maschera salvata per COUCH: ./prova/COUCH_mask.nii.gz
Elaborazione ROI: CTV 1
Maschera salvata per CTV 1: ./prova/CTV 1_mask.nii.gz
Elaborazione ROI: CTV 2
Maschera salvata per CTV 2: ./prova/CTV 2_mask.nii.gz
Elaborazione ROI: CTV 3
Maschera salvata per CTV 3: ./prova/CTV 3_mask.nii.gz
Elaborazione ROI: GTV
Maschera salvata per GTV: ./prova/GTV_mask.nii.gz
Elaborazione ROI: GTV PV
Maschera salvata per GTV PV: ./prova/GTV PV_mask.nii.gz
Elaborazione ROI: Larynx
Mascher

In [39]:
import pydicom

rtstruct = pydicom.dcmread(rtstruct_file)
contours = extract_contours(rtstruct)



ROI BODY: 115 contorni trovati.
ROI Brain: 57 contorni trovati.
ROI Brain Stem: 11 contorni trovati.
ROI Brain Stem PV: 13 contorni trovati.
ROI COUCH: 142 contorni trovati.
ROI CTV 1: 54 contorni trovati.
ROI CTV 2: 55 contorni trovati.
ROI CTV 3: 29 contorni trovati.
ROI GTV: 19 contorni trovati.
ROI GTV PV: 12 contorni trovati.
ROI Larynx: 11 contorni trovati.
ROI Mandibule: 35 contorni trovati.
ROI Mouth/CaviteOral: 20 contorni trovati.
ROI ring: 151 contorni trovati.
ROI PTV 1: 34 contorni trovati.
ROI PTV 2: 26 contorni trovati.
ROI PTV pres moelle: 24 contorni trovati.
ROI Parotid (L): 26 contorni trovati.
ROI Parotid (R): 26 contorni trovati.
ROI Parotid(R) - CTV: 22 contorni trovati.
ROI Spinal Cord: 35 contorni trovati.
ROI Spinal Cord +2mm: 36 contorni trovati.
ROI Spinal Cord + marge: 39 contorni trovati.
ROI TissueEquivalent: 60 contorni trovati.
ROI petit ring 70: 98 contorni trovati.
ROI CTV 3 bas: 25 contorni trovati.
ROI CTV 2 loin: 57 contorni trovati.
